In [92]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_community.vectorstores import FAISS

import faiss

import os
from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [34]:
books_list = os.listdir("books")
books_list

['as_a_man_thinketh.txt',
 'a_message_to_garcia.txt',
 'character_building.txt',
 'essays_by_ralph_waldo_emerson.txt',
 'how_to_live_on_24_hours_a_day.txt',
 'meditations.txt',
 'the_art_of_public_speaking.txt',
 'the_art_of_war.txt',
 'the_kybalion.txt',
 'the_power_of_concentration.txt',
 'the_science_of_getting_rich.txt',
 'the_way_to_wealth.txt']

### Text Splitting

In [42]:
raw_documents = []

In [43]:
for book in books_list:
    loader = TextLoader(os.path.join("books", book), encoding="utf-8")
    documents = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=["\n"])
    chunks = text_splitter.split_documents(documents)
    raw_documents.extend(chunks)

In [47]:
raw_documents[0].page_content

'AS A MAN THINKETH\n\n\nBY\n\nJAMES ALLEN\n\n\nAuthor of "From Passion to Peace"\n\n\n\n  _Mind is the Master power that moulds and makes,\n  And Man is Mind, and evermore he takes\n  The tool of Thought, and, shaping what he wills,\n  Brings forth a thousand joys, a thousand ills:--\n  He thinks in secret, and it comes to pass:\n  Environment is but his looking-glass._\n\n\n\nAuthorized Edition\n\nNew York\n\n\n\n\n\nCONTENTS\n\n\nTHOUGHT AND CHARACTER\n\nEFFECT OF THOUGHT ON CIRCUMSTANCES\n\nEFFECT OF THOUGHT ON HEALTH AND THE BODY\n\nTHOUGHT AND PURPOSE\n\nTHE THOUGHT-FACTOR IN ACHIEVEMENT\n\nVISIONS AND IDEALS\n\nSERENITY\n\n\n\n\n\n\nFOREWORD\n\n\nTHIS little volume (the result of meditation and experience) is not\nintended as an exhaustive treatise on the much-written-upon subject\nof the power of thought. It is suggestive rather than explanatory,\nits object being to stimulate men and women to the discovery and\nperception of the truth that--\n\n"They themselves are makers of th

In [56]:
raw_documents[1].page_content

'its object being to stimulate men and women to the discovery and\nperception of the truth that--\n\n"They themselves are makers of themselves."\n\nby virtue of the thoughts, which they choose and encourage; that\nmind is the master-weaver, both of the inner garment of character\nand the outer garment of circumstance, and that, as they may have\nhitherto woven in ignorance and pain they may now weave in\nenlightenment and happiness.\n\nJAMES ALLEN.\n\nBROAD PARK AVENUE,\n\nILFRACOMBE,\n\nENGLAND\n\n\n\n\n\nAS A MAN THINKETH\n\n\n\n\nTHOUGHT AND CHARACTER\n\n\nTHE aphorism, "As a man thinketh in his heart so is he," not only\nembraces the whole of a man\'s being, but is so comprehensive as to\nreach out to every condition and circumstance of his life. A man is\nliterally _what he thinks,_ his character being the complete sum of\nall his thoughts.\n\nAs the plant springs from, and could not be without, the seed, so\nevery act of a man springs from the hidden seeds of thought, and'

In [49]:
len(raw_documents)

3134

### Word Embeddings and Vector Database (using FAISS)

In [87]:
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
index_dir = "word_embeddings"

if os.path.exists(index_dir):
    vector_store = FAISS.load_local(index_dir, embeddings, allow_dangerous_deserialization=True)

else:
    raw_documents = []

    for book in books_list:
        loader = TextLoader(os.path.join("books", book), encoding="utf-8")
        documents = loader.load()
        chunks = text_splitter.split_documents(documents)
        raw_documents.extend(chunks)

    vector_store = FAISS.from_documents(raw_documents, embeddings)
    vector_store.save_local(index_dir)

In [90]:
query = "How to stay physically active?"
docs = vector_store.similarity_search_with_score(query, k=5)

for doc, score in docs:
    print(f"Score: {score}")
    print(doc.page_content)
    print("-" * 50)

Score: 0.9188239574432373
Exercise 1

Sit in a comfortable chair and see how still you can keep. This
is not as easy as it seems. You will have to center your
attention on sitting still. Watch and see that you are not making
any involuntary muscular movements. By a little practice you will
find you are able to sit still without a movement of the muscles
for fifteen minutes. At first I advise sitting in a relaxed
position for five minutes. After you are able to keep perfectly
still, increase the time to ten minutes and then to fifteen. This
is as long as it is necessary. But never strain yourself to keep
still. You must be relaxed completely. You will find this habit
of relaxing is very good.


Exercise 2

Sit in a chair with your head up and your chin out, shoulders
back. Raise your right arm until it is on the level with your
shoulder, pointing to your right. Look around, with head only,
and fix your gaze on your fingers, and keep the arm perfectly
------------------------------------

### Adding LLM Capabilities

In [99]:
llm = ChatGoogleGenerativeAI(model="models/gemini-2.5-flash")

In [107]:
user_query = "How can I build better mental determination?"

docs = vector_store.similarity_search_with_score(user_query, k=5)

context = "\n\n".join([doc.page_content for doc, score in docs])
print(context)

determination used determines who has the strongest will. No
one's is stronger than yours. Think of this whenever you go
against a strong opponent.

Never say "I can't concentrate today." You can do it just the
minute you say "I will." You can keep your thoughts from
straying, just the same as you can control your arms. When once
you realize this fact, you can train the will to concentrate on
anything you wish. If it wanders, it is your fault. You are not
utilizing your will. But, don't blame it on your will and say it
is weak. The will is just the same whether you act as if it were
weak or as if it were strong. When you act as if your will is
strong you say, "I can." When you act as if it were weak you say,
"I can't." It requires the same amount of effort, in each case.

Some men get in the habit of thinking "I can't" and they fail.
Others think "I can" and succeed. So remember, it is for you to
decide whether you will join the army of "I can't" or "I can."

possible. Once this has be

In [108]:
prompt = f"""
    You are a helpful self-help advisor.
    Answer the questions in clear, practical, and concise points.
    Use the following context from classic self-help books to guide your response.

    User's questions: {user_query}

    Relevant Documents: {context}

    Answer:
"""

In [109]:
response = llm.invoke(prompt)
print(response.content)

To build better mental determination, focus on these practical principles:

*   **Believe in Your Inherent Willpower:** Understand that your will is as strong as anyone's. The strength of your will is determined by how you choose to use it, not by an innate weakness.
*   **Adopt an "I Can" Mindset:** Consistently choose to think and act from a position of "I can," rather than "I can't." Both require effort, but the "I can" attitude leads to success.
*   **Take Responsibility for Your Focus:** Recognize that you have direct control over your thoughts and can prevent them from straying. If your mind wanders, it's a failure to utilize your will, not a sign that your will is weak.
*   **Practice Ceaseless Application and Perseverance:** Once you form a plan or set a goal, press its claim relentlessly until it's attained. Do not quit prematurely, as this weakens all future efforts.
*   **Cultivate Intense Concentration:** Direct every power of your mind and energy towards your undertaking. 